<a href="https://colab.research.google.com/github/adalves-ufabc/2021.QS-PLN/blob/main/2021_Q1_PLN_Notebook_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2020.QS]**
Prof. Alexandre Donizeti Alves

### **Modelagem de Tópicos com LDA**

Neste exemplo faremos a modelagem de tópicos no texto obtido de artigos da Wikipedia. Para baixar a biblioteca da `Wikipedia`, execute o seguinte comando:

In [1]:
!pip install wikipedia

Para visualizar nosso modelo de tópicos, usaremos a biblioteca `pyLDAvis`. Para fazer o download da biblioteca, execute o seguinte comando `pip`:

In [2]:
!pip install pyLDAvis==2.1.2

In [3]:
# scraping Wikipedia Articles

import wikipedia
import nltk

nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

global_warming = wikipedia.page("Natural language processing")
artificial_intelligence = wikipedia.page("Artificial Intelligence")
mona_lisa = wikipedia.page("Mona Lisa")
eiffel_tower = wikipedia.page("Eiffel Tower")

corpus = [global_warming.content, artificial_intelligence.content, mona_lisa.content, eiffel_tower.content]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


No script acima, primeiro importamos as bibliotecas `wikipedia` e `nltk`. Também baixamos as *stop words* em inglês. 

>
Em seguida, baixamos o artigo da Wikipedia especificando o tópico para o objeto `page` da biblioteca `wikipedia`. O objeto retornado contém informações sobre a página baixada.

>
Para recuperar o conteúdo da página Web, podemos usar o atributo `content`. O conteúdo de todos os quatro artigos é armazenado na lista denominada `corpus`.

**Pré-processamento dos dados**

Para realizar a modelagem de tópicos via LDA, precisamos de um dicionário de dados e da sacola de palavras (*bag of words*) do corpus. Para isso, precisamos de dados na forma de tokens.

>
Além disso, precisamos remover pontuações e *stop words* de nosso conjunto de dados. Por uma questão de uniformidade, converteremos todos os tokens para minúsculas e também os lematizaremos. Além disso, removeremos todos os tokens com menos de 5 caracteres.

In [4]:
import re
from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

def preprocess_text(document):
        # remove all the special characters
        document = re.sub(r'\W', ' ', str(document))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

        # substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        # converting to lowercase
        document = document.lower()

        # lemmatization
        tokens = document.split()
        tokens = [stemmer.lemmatize(word) for word in tokens]
        tokens = [word for word in tokens if word not in en_stop]
        tokens = [word for word in tokens if len(word)  > 5]

        return tokens

In [5]:
import nltk

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
processed_data = [];
for doc in corpus:
    tokens = preprocess_text(doc)
    processed_data.append(tokens)

No trecho de código acima nós iteramos através da lista de `corpus` que contém os quatro artigos da Wikipedia na forma de strings. Em cada iteração, passamos o documento para o método `preprocess_text` que criamos anteriormente. O método retorna tokens para esse documento específico. Os tokens são armazenados na lista `processed_data`.

>
No final do loop `for`, todos os tokens de todos os quatro artigos serão armazenados na lista `processed_data`. Agora podemos usar essa lista para criar um dicionário e a sacola de palavras correspondente ao corpus. O seguinte *script* faz isso:

In [7]:
from gensim import corpora

gensim_dictionary = corpora.Dictionary(processed_data)
gensim_corpus = [gensim_dictionary.doc2bow(token, allow_update=True) for token in processed_data]

A seguir, salvaremos nosso dicionário, bem como a sacola de palavras do corpus usando `pickle`. Usaremos o dicionário salvo mais tarde para fazer previsões sobre os novos dados.

In [8]:
import pickle

pickle.dump(gensim_corpus, open('gensim_corpus_corpus.pkl', 'wb'))
gensim_dictionary.save('gensim_dictionary.gensim')

Agora, temos tudo o que é necessário para criar o **modelo LDA** no `Gensim`. Usaremos a classe `LdaModel` do módulo `gensim.models.ldamodel` para criar o modelo LDA. Precisamos passar a sacola de palavras do corpus que criamos anteriormente como o primeiro parâmetro para o construtor `LdaModel`, seguido pelo número de tópicos, o dicionário que criamos anteriormente e o número de passagens (número de iterações para o modelo).

In [9]:
import gensim

lda_model = gensim.models.ldamodel.LdaModel(gensim_corpus, num_topics=4, id2word=gensim_dictionary, passes=20)
lda_model.save('gensim_model.gensim')

Sim, é assim tão simples. No script acima, criamos o modelo LDA de nosso conjunto de dados e o salvamos.

>
A seguir, vamos imprimir 10 palavras para cada tópico. Para fazer isso, podemos usar o método `print_topics`. Execute o seguinte *script*:

In [10]:
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.000*"eiffel" + 0.000*"intelligence" + 0.000*"second" + 0.000*"system" + 0.000*"painting" + 0.000*"artificial" + 0.000*"machine" + 0.000*"french" + 0.000*"leonardo" + 0.000*"learning"')
(1, '0.028*"eiffel" + 0.008*"second" + 0.007*"french" + 0.006*"structure" + 0.006*"exposition" + 0.005*"tallest" + 0.005*"engineer" + 0.005*"design" + 0.004*"million" + 0.004*"france"')
(2, '0.022*"painting" + 0.015*"language" + 0.010*"leonardo" + 0.008*"natural" + 0.006*"system" + 0.006*"machine" + 0.006*"louvre" + 0.005*"portrait" + 0.005*"learning" + 0.005*"sentence"')
(3, '0.020*"intelligence" + 0.015*"machine" + 0.014*"artificial" + 0.011*"system" + 0.011*"problem" + 0.008*"research" + 0.008*"knowledge" + 0.007*"approach" + 0.006*"learning" + 0.006*"computer"')


**IMPORTANTE**: A ordem dos tópicos pode mudar a cada execução do código.
>
O tópico 3 contém palavras como *language*, *natural*, *system*, *machine* etc. Podemos supor que essas palavras pertencem ao tópico relacionado a PLN.

Da mesma forma, o primeiro (0) contém palavras como *intelligence*, *machine*, *research* etc. Podemos supor que essas palavras pertencem ao tópico relacionado à Inteligência Artificial.

Podemos ver claramente que o modelo LDA identificou com sucesso os quatro tópicos em nosso conjunto de dados.

>
É importante mencionar aqui que o LDA é um algoritmo de aprendizado não supervisionado e, em problemas do mundo real, você não saberá sobre os tópicos do conjunto de dados de antemão. Você simplesmente receberá um corpus, os tópicos serão criados usando LDA e, em seguida, os nomes dos tópicos dependem de você.

Vamos agora criar 8 tópicos usando nosso conjunto de dados. Iremos imprimir 5 palavras por tópico:

In [11]:
lda_model = gensim.models.ldamodel.LdaModel(gensim_corpus, num_topics=8, id2word=gensim_dictionary, passes=15)
lda_model.save('gensim_model.gensim')
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.000*"intelligence" + 0.000*"artificial" + 0.000*"research" + 0.000*"machine" + 0.000*"problem"')
(1, '0.000*"language" + 0.000*"machine" + 0.000*"system" + 0.000*"intelligence" + 0.000*"natural"')
(2, '0.000*"intelligence" + 0.000*"eiffel" + 0.000*"painting" + 0.000*"machine" + 0.000*"language"')
(3, '0.000*"painting" + 0.000*"machine" + 0.000*"system" + 0.000*"intelligence" + 0.000*"language"')
(4, '0.013*"intelligence" + 0.012*"machine" + 0.012*"painting" + 0.010*"language" + 0.010*"system"')
(5, '0.000*"painting" + 0.000*"leonardo" + 0.000*"portrait" + 0.000*"language" + 0.000*"louvre"')
(6, '0.032*"eiffel" + 0.009*"second" + 0.008*"french" + 0.007*"structure" + 0.007*"exposition"')
(7, '0.000*"intelligence" + 0.000*"system" + 0.000*"eiffel" + 0.000*"language" + 0.000*"machine"')


Novamente, o número de tópicos que deseja criar depende de você. Continue tentando números diferentes até encontrar tópicos adequados. Para nosso conjunto de dados, o número adequado de tópicos é 4, pois já sabemos que nosso corpus contém palavras de quatro artigos diferentes. Reverta para quatro tópicos executando o seguinte *script*:

Desta vez, você verá resultados diferentes, uma vez que os valores iniciais para os parâmetros LDA são escolhidos aleatoriamente. Os resultados desta vez são os seguintes:

In [12]:
lda_model = gensim.models.ldamodel.LdaModel(gensim_corpus, num_topics=4, id2word=gensim_dictionary, passes=20)
lda_model.save('gensim_model.gensim')
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.000*"machine" + 0.000*"painting" + 0.000*"system" + 0.000*"intelligence" + 0.000*"language" + 0.000*"research" + 0.000*"artificial" + 0.000*"problem" + 0.000*"eiffel" + 0.000*"natural"')
(1, '0.020*"intelligence" + 0.015*"machine" + 0.014*"artificial" + 0.011*"system" + 0.011*"problem" + 0.008*"research" + 0.008*"knowledge" + 0.007*"approach" + 0.006*"learning" + 0.006*"computer"')
(2, '0.028*"eiffel" + 0.008*"second" + 0.007*"french" + 0.006*"structure" + 0.006*"exposition" + 0.005*"tallest" + 0.005*"engineer" + 0.005*"design" + 0.004*"million" + 0.004*"france"')
(3, '0.022*"painting" + 0.015*"language" + 0.010*"leonardo" + 0.008*"natural" + 0.006*"system" + 0.006*"machine" + 0.006*"louvre" + 0.005*"portrait" + 0.005*"learning" + 0.005*"sentence"')


**Avaliando o modelo LDA**

Conforme mencionado anteriormente, os modelos de aprendizagem não supervisionados são difíceis de avaliar, uma vez que não existe uma verdade concreta contra a qual possamos testar a saída de nosso modelo.

>
Suponha que temos um novo documento de texto e queremos encontrar seu tópico usando o modelo LDA que acabamos de criar, podemos fazer isso usando o seguinte *script*:

In [13]:
test_doc = 'Great structures are build to remember an event happened in the history.'
test_doc = preprocess_text(test_doc)
bow_test_doc = gensim_dictionary.doc2bow(test_doc)

print(lda_model.get_document_topics(bow_test_doc))

[(0, 0.08337962), (1, 0.08514082), (2, 0.7433581), (3, 0.08812153)]


No *script* acima, criamos uma string, criamos sua representação no dicionário e então convertemos a string no corpus do saco de palavras. A representação do saco de palavras é então passada para o método `get_document_topics`. 

A saída mostra que há 74,1% de chance de que o novo documento pertença ao tópico 3 (consulte as palavras para o tópico 3 na última saída). Da mesma forma, há 8,4% de chance de este documento pertencer ao primeiro tópico (0). Se olharmos para o tópico 3, ele contém palavras relacionadas à Torre Eiffel. Nosso documento de teste também contém palavras relacionadas a estruturas e edifícios. Portanto, foi atribuído o tópico 3.

Outra forma de avaliar o modelo LDA é por meio de `Perplexity` (Perplexidade) e `Coherence Score` (Coerência).

Como regra geral para um bom modelo de LDA, a pontuação de perplexidade deve ser baixa, enquanto a coerência deve ser alta. A biblioteca Gensim possui uma classe `CoherenceModel` que pode ser usada para encontrar a coerência do modelo LDA. Para perplexidade, o objeto `LdaModel` contém o método `log_perplexity` que pega uma sacola de palavras como parâmetro e retorna a perplexidade correspondente.

In [14]:
print('\nPerplexity:', lda_model.log_perplexity(gensim_corpus))

from gensim.models import CoherenceModel

coherence_score_lda = CoherenceModel(model=lda_model, texts=processed_data, dictionary=gensim_dictionary, coherence='c_v')
coherence_score = coherence_score_lda.get_coherence()

print('\nCoherence Score:', coherence_score)


Perplexity: -7.6300186146211155

Coherence Score: 0.552999173152782


**Visualizando o modelo LDA**

Para visualizar nossos dados, podemos usar a biblioteca `pyLDAvis` que baixamos no início.

In [15]:
gensim_dictionary = gensim.corpora.Dictionary.load('gensim_dictionary.gensim')
gensim_corpus = pickle.load(open('gensim_corpus_corpus.pkl', 'rb'))
lda_model = gensim.models.ldamodel.LdaModel.load('gensim_model.gensim')

import pyLDAvis.gensim

lda_visualization = pyLDAvis.gensim.prepare(lda_model, gensim_corpus, gensim_dictionary, sort_topics=False)
pyLDAvis.display(lda_visualization)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


Cada círculo na imagem acima corresponde a um tópico. A partir da saída do modelo LDA usando 4 tópicos, sabemos que o primeiro tópico está relacionado à Inteligência Artificial, o segundo tópico está relacionado ao PLN, o terceiro tópico está relacionado à Mona Lisa, enquanto o quarto tópico está relacionado à Torre Eiffel.

>
A distância entre os círculos mostra como os tópicos são diferentes uns dos outros. Você pode ver que os círculos 1 e 2 estão sobrepostos. Isso ocorre porque o tópico 1 (IA) e o tópico 2 (PLN) têm muitas palavras em comum.

Se você passar o mouse sobre qualquer palavra à direita, verá apenas o círculo do tópico que contém a palavra. Por exemplo, se você passar o mouse sobre a palavra "*machine*", verá que os tópicos 3 e 4 desaparecem, pois não contêm essa palavra. O tamanho do tópico 1 aumentará uma vez que a maioria das ocorrências da palavra "*machine*" estão dentro desse  tópico. Uma porcentagem menor está no tópico 2.

Da mesma forma, se você clicar em qualquer um dos círculos, uma lista dos termos mais frequentes para aquele tópico aparecerá à direita junto com a frequência de ocorrência naquele mesmo tópico.

**Mais informações:**

> https://stackabuse.com/python-for-nlp-working-with-the-gensim-library-part-2/